In [19]:
# conda: holo37

# cd to the 'lib' directory in your notebooks directory
# $ export LIBROOT=`pwd`
# Start jupyter lab from any directory

import sys, os
if (os.environ.get('LIBROOT')):
    sys.path.append(os.environ.get('LIBROOT'))

    from sysInfo import sysInfo
info = sysInfo()
info.show(vList=['platform','python','esmf','esmpy','xgcm','xesmf',
                 'netcdf4','numpy','xarray',
                 'cartopy','matplotlib',
                 'jupyter_core','jupyterlab','notebook',
                 'dask'])

# import gridUtils.py from local folder via LIBROOT path specified above
from gridUtils import gridUtils
import numpy as np
import cartopy.crs as ccrs
import panel as pn
import os, sys
import cartopy
import matplotlib.pyplot as plt
import netCDF4 as nc
import warnings
import xarray as xr
import xgcm
from io import BytesIO
warnings.filterwarnings('ignore')


Unable to show requested version information.  Please install 'conda' in this environment.
System name                             : Darwin
System name                             : james-imac
Operating system release                : 18.7.0
Operating system version                : Darwin Kernel Version 18.7.0: Tue Jan 12 22:04:47 PST 2021; root:xnu-4903.278.56~1/RELEASE_X86_64
Hardware identifier                     : x86_64
Active conda environment                : holo37
Unable to show requested version information.  Please install 'conda' in this environment.
Conda reported versions of software:
platform                                : not installed
python                                  : not installed
esmf                                    : not installed
esmpy                                   : not installed
xgcm                                    : not installed
xesmf                                   : not installed
netcdf4                                 : not installed


In [14]:
grd = gridUtils()
grd.verboseLevel = 1
grd.debugLevel = 1

grd.clearGrid()

# Specify the grid parameters
# nominalSpacing should be 2.0 for supergrid

# Former API
#grd.setGridParameters({
#    'gridProjection': 'Mercator',
#    'lonGridCenter': 230.0,
#    'latGridCenter': 40.0,
#    'lonSpan': 20.0,
#    'latSpan': 30.0,
#    'spanUnits': 'points',
#    'nominalResolution': 1.0,
#    'nominalSpacing': 2.0,
#    'gridTilt': 30.0
#})

# Lastest API
grd.setGridParameters({
    'projection': {
        'name': 'LambertConformalConic',
        'lon_0': 230.0,
        'lat_0': 40.0
    },
    'dx': 20.0,
    'dxUnits': 'degrees',
    'dy': 30.0,
    'dyUnits': 'degrees',
    'tilt': 30,
    'gridResolution': 1.0,
    'gridMode': 2.0
})

# Specify the desired resolution
refineR   = 1   # Inverse of nominal resolution
refineS   = 2   # Set to 2 for supergrid

projNames = ["Mercator", "Lambert Conformal Conic", "Polar Stereographic"]
projNamesGridTools = {
    'Mercator': 'Mercator',
    'Lambert Conformal Conic': 'LambertConformalConic',
    'Polar Stereographic': 'NorthPolarStereo'
}
projCarto = [ccrs.Mercator(), ccrs.LambertConformal(),  ccrs.NorthPolarStereo()]
projDict = dict(zip(projNames,projCarto))
projSel = pn.widgets.Select(name='Grid File Projection', options=projNames, value=projNames[1])

nomRes = pn.widgets.Spinner(name="Nominal Resolution", value=1, step=1, start=0, end=10, width=100)
nomSpace = pn.widgets.Spinner(name="Nominal Spacing", value=2, step=1, start=0, end=10, width=100)
unitNames = ['degrees','meters','points']
spanUnits = pn.widgets.Select(name='Units', options=unitNames, value=unitNames[2])
lon_span = pn.widgets.Spinner(name="Longitude Span", value=30, step=1, start=0, end=100, width=100)
lat_span = pn.widgets.Spinner(name="Latitude Span", value=30, step=1, start=0, end=100, width=100)
lon0 = pn.widgets.Spinner(name="Central Longitude (0 to 360)", value=300, step=1, start=0, end=360, width=100)
lat0 = pn.widgets.Spinner(name="Central Latitude (-90 to 90)", value=40, step=1, start=-90, end=90, width=100)
tilt = pn.widgets.Spinner(name="Tilt (-90 to 90)", value=30, step=1, start=-90, end=90, width=100)

In [15]:
def make_plot(lon0, lon_span, lat0, lat_span, tilt, projSel, nomRes, nomSpace, spanUnits):
    grd.clearGrid()
    mp_title = "Nearside Perspective: " + str(lon_span) + "x" + str(lat_span) + " with " + str(tilt) + " degree tilt"
    print(projSel)
    
    refineR   = 1   # Inverse of nominal resolution
    refineS   = 2   # Set to 2 for supergrid
    
    grd.setGridParameters({
        'projection': {
            'name': projNamesGridTools[projSel],
            'lon_0': lon0,
            'lat_0': lat0
        },
        'dx': lon_span,
        'dy': lat_span,
        'dxUnits': 'degrees',
        'dyUnits': 'degrees',
        'gridResolution': nomRes,
        'gridMode': nomSpace,
        'tilt': tilt
    })
    grd.makeGrid()
    
    grd.setPlotParameters(
        {
            'figsize': (12,12),
            'projection' : {
                'name': 'NearsidePerspective',
                'lat_0': lat0,
                'lon_0': lon0
            },
            'extent': [lon0 - lon_span , lon0 + lon_span, lat0 - lat_span, lat0 + lat_span],
            'iLinewidth': 1.0,
            'jLinewidth': 1.0,
            'showGridCells': True,
            'title': mp_title,
            'iColor': 'k',
            'jColor': 'k'
        }
    )
    
    (figure, axes) = grd.plotGrid()
    # Suppress showing figure in a notebook cell outside the panel/bokeh/geoview application.
    # This still allows the figure object to be passed around and to the application.
    plt.close()
    print(figure)
    
    return figure

def make_grid(lon0, lon_span, lat0, lat_span, tilt, projSel, nomRes, nomSpace, spanUnits):
    grd.clearGrid()
    #refineR   = 1   # Inverse of nominal resolution
    #refineS   = 2   # Set to 2 for supergrid
    # Specify the grid parameters
    # nominalSpacing should be 2.0 for supergrid
    grd.setGridParameters({
        'projection': {
            'name': projSel,
            'lon_0': lon0,
            'lat_0': lat0
        },
        'dx': lon_span,
        'dy': lat_span,
        'dxUnits': 'degrees',
        'dyUnits': 'degrees',    
        'gridResolution': nomRes,
        'gridMode': nomSpace,
        'tilt': tilt
    })
    grd.makeGrid()
    
    return


In [16]:
import xarray as xr

def test_grid(lon0, lon_span, lat0, lat_span, tilt, projSel, nomRes, nomSpace, spanUnits):
    mom6_grid = xr.open_dataset("/Users/james/Documents/Github/esm_lab/gridGen/mom6_xgcm_grid.nc")
    return mom6_grid

In [20]:
def download_cb():
    bout = test_grid(lon0,lon_span,lat0,lat_span,tilt, projSel, nomRes, nomSpace, spanUnits).to_netcdf()
    bio = BytesIO()
    bio.write(bout)
    bio.seek(0)
    return bio

# create file download button
fd = pn.widgets.FileDownload(
    callback=download_cb,
    filename='gridFile.nc')

def update(event):
    plotwindow[1].object = make_plot(lon0.value,lon_span.value,lat0.value,lat_span.value,tilt.value, projSel.value, nomRes.value, nomSpace.value, spanUnits)


generate_button = pn.widgets.Button(name='Plot', button_type='primary')
generate_button.on_click(update)


#header = pn.Row(pn.panel(mom6_logo, width=170),  pn.layout.Spacer(width=10), 
#                pn.Column(pn.Pane(title1, width=1000), pn.Pane(instruction, width=1000)))


sel_box = pn.WidgetBox(lon_span, lat_span, lon0, lat0, tilt, projSel, nomRes, nomSpace, spanUnits, generate_button, pn.layout.Spacer(height=30), fd)

plotwindow = pn.Row(sel_box, make_plot(lon0.value,lon_span.value,lat0.value,lat_span.value,tilt.value, projSel.value, nomRes.value, nomSpace.value, spanUnits), 
                    test_grid(lon0.value,lon_span.value,lat0.value,lat_span.value,tilt.value, projSel.value, nomRes.value, nomSpace.value, spanUnits))
dashboard = pn.Column(plotwindow)
dashboard.show()


Lambert Conformal Conic
Generating regular lat-lon grid centered at 300.00 0.00 on equator.
   Generated regular lat-lon grid between latitudes -15.00 15.00
   Number of js=61
Figure(864x864)
Launching server at http://localhost:52848
